In [71]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [72]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

## Request the Falcon9 Launch from Wikipedia

In [73]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [74]:
response = requests.get(static_url)

In [75]:
soup = BeautifulSoup(response.text, "html.parser")

In [76]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

## Extract data from HTML table

In [77]:
html_tables = soup.find_all("table")
first_launch_table = html_tables[2]

In [78]:
# print(first_launch_table)

## Create a Dataframe parsing the HTML

In [79]:
column_names = []

headers = first_launch_table.find_all("th")

for th in headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [80]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [81]:
extracted_row = 0
#Extract each table 
for table_number, table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag = False
        #get table element 
        row = rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict["Flight No."].append(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
              
            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            #print(payload_mass)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else 'N/A'
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [82]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [83]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [84]:
df['Booster landing'].value_counts()

Booster landing
Success         80
No attempt      18
Failure         10
Controlled       5
No attempt\n     4
Uncontrolled     2
Failure          1
Precluded        1
Name: count, dtype: int64

In [85]:
df['Launch outcome'] = df['Booster landing'].str.strip()

df['Class'] = df['Launch outcome'].apply(lambda x: 1 if x in ['Success', 'Controlled'] else 0)

df['Date'] = pd.to_datetime(df['Date'], format='%d %B %Y')

df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

print(df[['Launch outcome', 'Class']])

    Launch outcome  Class
0          Failure      0
1          Failure      0
2       No attempt      0
3       No attempt      0
4       No attempt      0
..             ...    ...
116        Success      1
117        Success      1
118        Success      1
119        Success      1
120        Success      1

[121 rows x 2 columns]


In [86]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Class
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Failure,F9 v1.07B0003.18,Failure,2010-06-04,18:45,0
1,2,CCAFS,Dragon,0,LEO,NASA,Failure,F9 v1.07B0004.18,Failure,2010-12-08,15:43,0
2,3,CCAFS,Dragon,525 kg,LEO,NASA,No attempt,F9 v1.07B0005.18,No attempt\n,2012-05-22,07:44,0
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,No attempt,F9 v1.07B0006.18,No attempt,2012-10-08,00:35,0
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,No attempt,F9 v1.07B0007.18,No attempt\n,2013-03-01,15:10,0


In [87]:
df.to_csv('data/spacex_web_scraped.csv', index=False)